In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
#import pandasql as psql
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
#imp = SimpleImputer(missing_values=np.nan, strategy='mean') #with mean accuracy 79.354 . median rediced the accuracy
imp2=SimpleImputer(strategy="most_frequent")
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
train=pd.read_csv('../input/spaceship-titanic/train.csv')
test=pd.read_csv('../input/spaceship-titanic/test.csv')
print(train.shape, test.shape)

In [ ]:
train[['deck','srnum','side']] = train['Cabin'].str.split('/',expand=True)
test[['deck','srnum','side']]  = test['Cabin'].str.split('/',expand=True)
train.drop(['Cabin','Name'],axis=1,inplace=True)
test.drop(['Cabin','Name'],axis=1,inplace=True)

#--- missing values imputation for categorical cols---#

charcols = ['HomePlanet','CryoSleep','side','deck']

numcols =  ['Age','RoomService','VRDeck','ShoppingMall','FoodCourt','Spa']

imp2.fit(train[charcols])
#train[numcols] = imp.fit_transform(train[numcols])
train[charcols] = imp2.fit_transform(train[charcols])
#test[numcols] = imp.transform(test[numcols])
test[charcols] = imp2.transform(test[charcols])

#--label encoding char columns, accuracy 80.005--#

le_dict = {col: LabelEncoder() for col in charcols }
for col in charcols:
    train[col] =le_dict[col].fit_transform(train[col])
    test[col]=le_dict[col].transform(test[col])
#newtrain.head(50)
#newtrain.deck.value_counts() '''
#le_dict


In [ ]:
#train.describe()
train1=train[train['CryoSleep'] ==1]
train2=train[train['CryoSleep'] ==0]
test1=test[test['CryoSleep'] ==1]
test2=test[test['CryoSleep'] ==0]
print(train1.shape,train2.shape,test1.shape, test2.shape)



In [ ]:
train1['RoomService'].fillna(0,inplace=True)
test1['RoomService'].fillna(0,inplace=True)
train2['RoomService'].fillna(train2['RoomService'].median(),inplace=True)
test2['RoomService'].fillna(test2['RoomService'].median(),inplace=True)
                                  
train1['FoodCourt'].fillna(0,inplace=True)
test1['FoodCourt'].fillna(0,inplace=True)
train2['FoodCourt'].fillna(train2['FoodCourt'].median(),inplace=True)
test2['FoodCourt'].fillna(test2['FoodCourt'].median(),inplace=True)

train1['ShoppingMall'].fillna(0,inplace=True)
test1['ShoppingMall'].fillna(0,inplace=True)
train2['ShoppingMall'].fillna(train2['ShoppingMall'].median(),inplace=True)
test2['ShoppingMall'].fillna(test2['ShoppingMall'].median(),inplace=True)

train1['Spa'].fillna(0,inplace=True)
test1['Spa'].fillna(0,inplace=True)
train2['Spa'].fillna(train2['Spa'].median(),inplace=True)
test2['Spa'].fillna(test2['Spa'].median(),inplace=True)

train1['VRDeck'].fillna(0,inplace=True)
test1['VRDeck'].fillna(0,inplace=True)
train2['VRDeck'].fillna(train2['VRDeck'].median(),inplace=True)
test2['VRDeck'].fillna(test2['VRDeck'].median(),inplace=True)

In [ ]:
train=pd.concat([train1,train2])
test=pd.concat([test1,test2])

In [ ]:
train['Age'].fillna(train['Age'].median(),inplace=True)  #mean
test['Age'].fillna(test['Age'].median(),inplace=True)

In [ ]:
dict = {True:1,False:0}
train['Transported'] = train['Transported'].map(dict)
y = train['Transported']

In [ ]:
#y

In [ ]:
cols = ['HomePlanet','CryoSleep','side','deck','Age','RoomService','VRDeck','ShoppingMall','FoodCourt','Spa']
newtrain=train[cols]
newtest = test[cols]

In [ ]:
#---use one hot encoding----#
#newtrain=pd.get_dummies(data=newtrain,columns =charcols)
#newtest=pd.get_dummies(data=newtest,columns =charcols)
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#newtrain[numcols] = pd.DataFrame(scaler.fit_transform(newtrain[numcols]))
#newtest[numcols] = pd.DataFrame(scaler.transform(newtest[numcols]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(newtrain,y, train_size=0.8,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model =RandomForestClassifier(n_estimators=150,criterion='gini',max_depth=100,max_leaf_nodes=120,verbose=1)
#model =RandomForestClassifier(n_estimators=50,criterion='gini',max_depth=50,max_leaf_nodes=120,verbose=1)
model.fit(X_train, y_train)
preds=model.predict(X_valid)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_valid, preds)
cnf_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')


from sklearn.metrics import classification_report
print(classification_report(y_valid, preds))


In [ ]:
feats = {} # a dict
for feature, importance in zip(newtrain.columns, model.feature_importances_):
    feats[feature] = importance 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').plot(kind='bar', rot=45)


In [ ]:
# predict on test values
preds=model.predict(newtest)
preds

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': preds})

In [ ]:
dict3 = {1:True,0:False}
output['Transported'] = output['Transported'].map(dict3)
output[:4]

In [ ]:
output.to_csv('./submission.csv', index=False)

In [ ]:
model2= LGBMClassifier(random_state=0,learning_rate=0.1,max_depth=5,n_estimators=100 )
model2.fit(X_train,y_train)
model2.score(X_valid,y_valid)

In [ ]:
pred=model2.predict(newtest)

In [ ]:
pred

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': pred})
dict3 = {1:True,0:False}
output['Transported'] = output['Transported'].map(dict3)
output[:4]